# Deploy disease detection model to ROS2

In the last section, we trained and evaluated YOLOv7 model on the PlantDoc dataset. Next, we will deploy the trained disease detection model to Robot Operating System 2 (ROS2)

## So What is ROS2

Robot Operating System (ROS) is a set of open-source software libraries and tools for building robotic systems. It provides a common framework for creating, programming, and deploying robotic systems in various domains, including industrial, research, and personal use.

ROS2 is the second major version of ROS. It was designed to address some of the limitations of the original ROS and to improve performance, scalability, and security. Some of the main features of ROS2 include:

- Real-time support: ROS2 supports real-time computing with the use of the DDS (Data Distribution Service) middleware, which allows for low-latency communication between ROS2 nodes.
- Improved scalability: ROS2 allows for distributed systems, with multiple computers communicating over a network. This allows for better scalability and the ability to handle more complex robotic systems.
- Better security: ROS2 uses DDS security features, such as authentication and encryption, to provide better security for robotic systems.
- Improved support for multiple languages: ROS2 supports multiple languages, including C++, Python, and C#, which allow the developers to use the language they are most comfortable with.
- Robustness: ROS2 improve robustness of the system, as well as resilience to communication failures and other unexpected events.

There are several core concepts that are fundamental to understanding how ROS2 works:

1. Nodes: In ROS2, nodes are the basic units of computation. A node represents a process that runs on a computer and communicates with other nodes. Nodes can be written in any programming language that is supported by ROS2, such as C++, Python, or C#. Each node can provide or consume one or more services or publish and subscribe to one or more topics.
2. Topics: Topics are the means by which nodes can exchange messages. In ROS2, a topic is a named channel to which nodes can publish messages and to which other nodes can subscribe to receive messages. Topics are used to send sensor data, control commands, and other types of information between nodes.
3. Services: Services provide a mechanism for nodes to request and receive a specific piece of information or to perform a specific action. Services are synchronous and request-response based, which means that a client node sends a request to a service and waits for a response. Services in ROS2 are defined by a pair of request and response message types.
4. Actions: Actions are similar to services but are designed for more complex tasks with feedback and goal status. They can be used to control the execution of long-lasting tasks, such as navigation or grasping.


To deploy the trained disease detection YOLOv7 model to ROS2, we need take the following steps:

- Install ROS2 and the required dependencies. 
- Write a ROS2 node to simulate a camera to capture the plant disease images, and use the ROS2 API to publish the images as messages on a ROS2 topic.
- Write a ROS2 node to subscribe to the images topic and perform plant disease detection on the received images.

## Write a node to publish images

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
import cv2
import numpy as np
import os

class MinimalPublisher(Node):
    def __init__(self):
        super().__init__('minimal_publisher')
        self.publisher = self.create_publisher(Image, 'Image', 10)
        timer_period = 0.02 # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)

        root_dir = "PlantDoc-1/test/images/"
        self.img_paths = [
            os.path.join(root_dir, img_name)
            for img_name in os.listdir(root_dir)
            ]
        self.n_imgs = len(self.img_paths)

        self.cv_image = cv2.imread(self.img_paths[0]) # the first image
        self.i = 1
        self.bridge = CvBridge()

    def timer_callback(self):
        self.publisher.publish(self.bridge.cv2_to_imgmsg(self.cv_image[:,:416]))
        self.get_logger().info('Publishing an image')

        height, width, channels = self.cv_image.shape
        if width < 416 + 100: # concat the next image
            next_cv_image = cv2.imread(self.img_paths[self.i])
            self.cv_image = cv2.hconcat([self.cv_image, next_cv_image])
            self.i += 1
            if self.i == self.n_imgs:
                self.i = 0
        self.cv_image = self.cv_image[:, 1:]


def main(args=None):
    rclpy.init(args=args)
    minimal_publisher = MinimalPublisher()
    rclpy.spin(minimal_publisher)
    minimal_publisher.destroy_node()
    rclpy.shutdown()

if __name__ == "__main__":
    main()

Looking at the code above, it can mainly be broken down into 3 parts: initialization, timer callback, and starting the node.

### Initialization
In the initialization step, we declare a node called `minimal_publisher`. Next we create a basic publisher that utilize the `sensor_msgs.msg.Image` type to publish a topic name `Image` per `timer_period = 0.02` second. Next we prepare all the test images in `img_paths`. Then we also create a basic `CvBridge` tool to convert images from an OpenCV compatible type to a ROS2 `sensor_msgs.msg.Image` message type.

### Timer callback
Within the callback `timer_callback`, the images are converted to the ROS2 message type using `CvBridge` sequentially. We concatenate the images one by one.

### Starting the node
In the `main` function, the node is initialized and spined up to publish images.

## Write a node to subscribe images and run plant disease detection on the images

In [ ]:
import rclpy
from rclpy.node import Node
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
import cv2

import torch
import numpy as np
from numpy import random
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression, scale_coords, set_logging
from utils.plots import plot_one_box
from utils.torch_utils import select_device, time_synchronized, TracedModel


class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('image_subscriber')
        self.subscription = self.create_subscription(Image, 'Image', self.listener_callback, 10)
        self.subscription # prevent unused variable warning
        self.bridge = CvBridge()

        device = 'cuda:0'
        set_logging()
        device = select_device('0')
        self.device = device
        half = device.type != 'cpu'
        self.half = half
        weights = 'best.pt'
        imgsz = 416
        model = attempt_load(weights, map_location=device)
        stride = int(model.stride.max())
        imgsz = check_img_size(imgsz, s=stride)
        model = TracedModel(model, device, img_size=imgsz)
        model.half()
        names = model.module.names if hasattr(model, 'module') else model.names
        self.names = names
        print(names)
        colors = [[random.randint(0,255) for _ in range(3)] for _ in names]
        self.colors = colors
        if device.type != 'cpu':
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))
        self.model = model

        fourcc = cv2.VideoWriter.fourcc('m', 'p', '4', 'v')
        self.out = cv2.VideoWriter('output.mp4', fourcc, 50, (416, 416))


    def listener_callback(self, data):
        self.get_logger().info('Receiving image')
        img0 = self.bridge.imgmsg_to_cv2(data)
        self.get_logger().info(str(img0.shape))
        img = img0[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img.half() if self.half else img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        with torch.no_grad():
            pred = self.model(img, augment=False)[0]
        conf_thres = 0.1
        iou_thres = 0.45
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)
        for i, det in enumerate(pred):  # detections per image
            s = ""
            gn = torch.tensor(img0.shape)[[1,0,1,0]]
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
                for *xyxy, conf, cls in reversed(det):
                    label = f'{self.names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=self.colors[int(cls)], line_thickness=1)
        # cv2.imshow("IMAGE", img0)
        # cv2.waitKey(1)
        # cv2.destroyWindow("IMAGE")
        cv2.imwrite('received_test.jpg', img0)
        self.out.write(img0)

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()
    rclpy.spin(image_subscriber)
    image_subscriber.destroy_node()
    rclpy.shutdown()

if __name__ == "__main__":
    main()


Our results are recorded as a short video.

In [2]:
%%html
<iframe width="393" height="698" src="https://www.youtube.com/embed/ty6OHxJW4tk" title="Realtime plant disease detection using YOLOv7 and ROS2" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>